In [4]:
import networkx as nx
import torch
from torch_geometric.data import Data
import random
import itertools
import matplotlib.pyplot as plt
import pickle

In [73]:
def generate_regular_biconnected_graph(d, n, max_tries=10):
    """
    Generate a d-regular biconnected graph with n nodes.
    """
    for attempt in range(max_tries):
        try:
            G = nx.random_regular_graph(d, n)
            if nx.is_biconnected(G):
                return G
        except nx.exception.NetworkXError:
            continue
    raise ValueError(f"Failed to generate a {d}-regular biconnected graph with {n} nodes after {max_tries} tries.")

def find_disjoint_edge_pairs(G, num_pairs):
    """
    Find 'num_pairs' disjoint edges in G.
    Returns a list of tuples representing the edge pairs.
    """
    edges = list(G.edges())
    random.shuffle(edges)
    selected_pairs = []
    used_nodes = set()
    for u, v in edges:
        if u not in used_nodes and v not in used_nodes:
            selected_pairs.append((u, v))
            used_nodes.update([u, v])
            if len(selected_pairs) == num_pairs:
                break
    if len(selected_pairs) < num_pairs:
        raise ValueError(f"Not enough disjoint edge pairs available. Needed: {num_pairs}, Found: {len(selected_pairs)}")
    return selected_pairs

def generate_single_graph(degree, num_components, max_nodes_per_component, max_tries=5):
    """
    Generate a single regular graph structured as a tree of regular biconnected components.
    """
    for attempt in range(max_tries):
        try:
            G = nx.Graph()
            tree = nx.random_unlabeled_tree(num_components)
            
            # Assign unique global node IDs
            current_node_id = 0
            components_nodes = []
            component_available_nodes = {}
            connector_nodes = []
            cut_edges = []
            
            # The degree for each component based on the tree
            component_degrees = dict(tree.degree())
            
            for component_id in range(num_components):
                connections_required = component_degrees[component_id]
                
                # Minimum number of nodes required to accommodate connector nodes
                min_nodes_i = connections_required * (degree - 1)
                min_nodes_i = max(min_nodes_i, degree)
                
                if min_nodes_i > max_nodes_per_component:
                    raise ValueError(f"Component {component_id}: Required minimum nodes {min_nodes_i} exceeds max_nodes_per_component {max_nodes_per_component}.")
                
                n_i = random.randint(min_nodes_i, max_nodes_per_component)
                if (n_i * degree) % 2 != 0:
                    if n_i < max_nodes_per_component:
                        n_i += 1
                    else:
                        n_i -= 1
                    if (n_i * degree) % 2 != 0 or n_i < min_nodes_i:
                        raise ValueError(f"Component {component_id}: Cannot adjust number of nodes to satisfy regularity.")
                
                component_graph = generate_regular_biconnected_graph(degree, n_i)
                
                # Relabel nodes to have unique global IDs
                mapping = {node: node + current_node_id for node in component_graph.nodes()}
                component_graph = nx.relabel_nodes(component_graph, mapping)
                
                G = nx.compose(G, component_graph)
                component_node_list = list(component_graph.nodes())
                components_nodes.append(component_node_list)
                component_available_nodes[component_id] = set(component_node_list)
                current_node_id += n_i
            
            # process edges to connect components
            for edge in tree.edges():
                u, v = edge  # Component indices
                num_pairs = (degree - 1)//2
                
                # Connector for component u
                c_u = current_node_id
                current_node_id += 1
                # Select d-1 available nodes from component u
                available_u = list(component_available_nodes[u])
                if len(available_u) < (degree - 1):
                    raise ValueError(f"Component {u}: Not enough available nodes to connect connector node.")
                
                pairs_u = find_disjoint_edge_pairs(G.subgraph(available_u), num_pairs)
                for (node1, node2) in pairs_u:

                    G.remove_edge(node1, node2)
                    G.add_edge(c_u, node1)
                    G.add_edge(c_u, node2)
                    component_available_nodes[u].remove(node1)
                    component_available_nodes[u].remove(node2)
                
                # Connector for component v
                c_v = current_node_id
                current_node_id += 1
                # Select d-1 available nodes from component v
                available_v = list(component_available_nodes[v])
                if len(available_v) < (degree - 1):
                    raise ValueError(f"Component {v}: Not enough available nodes to connect connector node.")
                
                pairs_v = find_disjoint_edge_pairs(G.subgraph(available_v), num_pairs)
                for (node1, node2) in pairs_v:
                    G.remove_edge(node1, node2)
                    G.add_edge(c_v, node1)
                    G.add_edge(c_v, node2)
                    component_available_nodes[v].remove(node1)
                    component_available_nodes[v].remove(node2)

                
                # Connect the two connector nodes
                G.add_edge(c_u, c_v)
                cut_edges.extend([(c_u, c_v), (c_v, c_u)])
                # Mark as cut vertices
                connector_nodes.extend([c_u, c_v])
            
            for node, deg in G.degree():
                if deg != degree:
                    
                    raise ValueError(f"Node {node} has degree {deg}, expected {degree}.")
            
            return G, connector_nodes, cut_edges
        except ValueError as e:
            if attempt < max_tries - 1:
                continue
            else:
                raise e

def transform_to_torch_geometric_data(G, cut_vertices, cut_edges):
    """
    Transform a networkx graph into a torch_geometric.data.Data object.
    """
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    x = torch.tensor([[i] for i in range(num_nodes)], dtype=torch.long)
    
    # Initialize labels
    y = torch.zeros(121, dtype=torch.long) # Use 121 for compatibility
    y[cut_vertices] = 1
    
    # Get edge list
    edge_index = torch.tensor(list(G.edges()), dtype=torch.long).t().contiguous()
    edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

    # Initialize edge labels
    edge_labels = torch.zeros(304, dtype=torch.float) # Hard code 304 for compatibility
    # Check for cut edges
    for idx in range(num_edges*2):
        if (edge_index[0, idx], edge_index[1, idx]) in cut_edges:
            edge_labels[idx] = 1
    
    data = Data(x=x, edge_index=edge_index, y=y, edge_labels=edge_labels)
    return data

def generate_dataset(num_graphs, degree, num_components, max_nodes_per_component):
    """
    Generate a dataset of torch_geometric.data.Data objects with cut vertices and cut edges.
    """
    dataset = []
    for graph_idx in range(num_graphs):
        try:
            G, cut_vertices, cut_edges = generate_single_graph(
                degree=degree,
                num_components=num_components,
                max_nodes_per_component=max_nodes_per_component
            )
            data = transform_to_torch_geometric_data(G, cut_vertices, cut_edges)
            dataset.append(data)
        except ValueError as e:
            pass
    return dataset



In [74]:
rand_dataset = []
max_nodes_per_component = 10  # Must be >= degree
num_graphs = 123  # Number of graphs in the dataset, total ~580. ideally 116 per num_cmpn

degree_list = [5, 3, 5, 3, 3] # random sequence of 3 and 5
# Generate dataset
for num_cmpn in range(3, 8):
    dataset = generate_dataset(
    num_graphs=num_graphs,
    degree=degree_list[num_cmpn - 3],
    num_components=num_cmpn,
    max_nodes_per_component=max_nodes_per_component
    )
    rand_dataset.extend(dataset)

if len(rand_dataset) > 0:
    graph = rand_dataset[0]
    print("Graph Information:")
    print(f"Number of nodes: {graph.num_nodes}")
    print(f"Number of edges: {graph.num_edges // 2}")  # Since edges are duplicated
    #print(f"Node features (x):\n{graph.x}")
    print(f"shape of x: {graph.x.shape}")
    print(f"first elements of x: {graph.x[:5]}")
    print(f"Node labels (y):\n{graph.y}")
    print(f"graph: {graph}")
    print(f"edge index: {graph.edge_index}")
    print(f"edge labels: {graph.edge_labels}")
    print(f"Total graphs generated: {len(rand_dataset)}")
else:
    print("No graphs were generated.")

Graph Information:
Number of nodes: 26
Number of edges: 65
shape of x: torch.Size([26, 1])
first elements of x: tensor([[0],
        [1],
        [2],
        [3],
        [4]])
Node labels (y):
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
graph: Data(x=[26, 1], edge_index=[2, 130], y=[121], edge_labels=[304])
edge index: tensor([[ 0,  0,  0,  0,  0,  1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  4,  4,  6,
          6,  6,  6,  6,  7,  7,  7,  7,  8,  8,  8,  9,  9,  9,  9, 10, 10, 10,
         11, 11, 11, 14, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17,
         18, 18, 18, 18, 19, 19, 20, 20, 21, 22, 24,  1,  4,  3,  

In [75]:
max_edges = 0
for graph in rand_dataset:
    num_edges = graph.edge_index.shape[1]
    if num_edges > max_edges:
        max_edges = num_edges
print(max_edges)

290


In [76]:
def build_g1_edges(m, k):
    """
    Build the undirected edges for G1 in 0-based indexing.
    Returns a list of (i, j) with i <-> j.
    """
    n = 2 * k * m + 1
    edges = []
    
    # if i0 < 2km-1, next = i0+1, else next=0 => forms a cycle.
    num_cycle_nodes = 2 * k * m  # = 2km
    for i0 in range(num_cycle_nodes):
        j0 = (i0 + 1) % num_cycle_nodes
        edges.append((i0, j0))
    
    n0 = num_cycle_nodes  # 2km
    for i in range(1, num_cycle_nodes + 1):
        if i % m == 0:
            i0 = i - 1  # 0-based
            edges.append((n0, i0))
    
    undirected = []
    for (a, b) in edges:
        undirected.append((a, b))
        undirected.append((b, a))
    return undirected

def build_g2_edges(m, k):
    """
    Build the undirected edges for G2 in 0-based indexing.
    """
    n = 2 * k * m + 1
    edges = []
    
    num_cycle = k * m
    for i0 in range(num_cycle):
        j0 = (i0 + 1) % num_cycle
        edges.append((i0, j0))
    
    for i0 in range(num_cycle):
        # shift by km
        shifted_i0 = i0 + num_cycle
        shifted_j0 = ((i0 + 1) % num_cycle) + num_cycle
        edges.append((shifted_i0, shifted_j0))
    
    n0 = 2 * k * m
    for i in range(1, 2 * k * m + 1):
        if i % m == 0:
            i0 = i - 1
            edges.append((n0, i0))
    
    undirected = []
    for (a, b) in edges:
        undirected.append((a, b))
        undirected.append((b, a))
    return undirected

def make_edge_index_and_labels(edges, max_edges=152):
    """
    Given a list of directed edges (i, j), get edge_index with shape [2, E] and edge-labels of 0s.
    """
    edges = list(dict.fromkeys(edges))  # preserves order
    E = len(edges)

    edge_index = torch.zeros((2, E), dtype=torch.long)
    for idx, (i, j) in enumerate(edges):
        edge_index[0, idx] = i
        edge_index[1, idx] = j
    
    edge_labels = torch.zeros(304, dtype=torch.long)
    return edge_index, edge_labels, E

def set_labels_for_G2(y, edge_labels, edges, m, k):
    """
    Set edge labels for G2 family based on whether edge is cut edge.
    """
    # Careful when converting 0 and 1 based nodes
    n_1 = 2*m*k
    if k != 1:
        y[n_1] = 1
    else:
        y[m - 1] = 1
        y[2*m - 1] = 1
        y[n_1] = 1
        
        special_pairs = [
            (m - 1, n_1),
            (2*m - 1, n_1),
            (n_1, m - 1),
            (n_1, 2*m - 1),
        ]
        edge_to_index = {}
        for idx, (src, dst) in enumerate(edges):
            edge_to_index[(src, dst)] = idx
        
        for (a, b) in special_pairs:
            if (a, b) in edge_to_index:
                idx = edge_to_index[(a, b)]
                if idx < 304:  # safety check
                    edge_labels[idx] = 1

# Whole family construction
def build_family_c9():
    familyc9_1 = []
    familyc9_2 = []
    
    max_num_nodes = 121  # largest possible n = 2*4*15 + 1
    for m in range(1, 20):
        for k in range(1, 20):
            if m * k < 3 or m * k > 60:
                continue
            n = 2 * k * m + 1
            # G1
            edges_g1 = build_g1_edges(m, k)
            edge_index_g1, edge_labels_g1, E1 = make_edge_index_and_labels(edges_g1)
            x_g1 = torch.tensor([[i] for i in range(n)], dtype=torch.long)
            # y: length 289, all zeros for G1
            y_g1 = torch.zeros(max_num_nodes, dtype=torch.long)
            data_g1 = Data(
                x=x_g1,
                edge_index=edge_index_g1,
                y=y_g1,
                edge_labels=edge_labels_g1
            )
            familyc9_1.append(data_g1)
            
            # G2
            edges_g2 = build_g2_edges(m, k)
            edge_index_g2, edge_labels_g2, E2 = make_edge_index_and_labels(edges_g2)
            
            x_g2 = torch.tensor([[i] for i in range(n)], dtype=torch.long)
            y_g2 = torch.zeros(max_num_nodes, dtype=torch.long)
            
            # Label cut edges
            set_labels_for_G2(y_g2, edge_labels_g2, edges_g2, m, k)
            data_g2 = Data(
                x=x_g2,
                edge_index=edge_index_g2,
                y=y_g2,
                edge_labels=edge_labels_g2
            )
            familyc9_2.append(data_g2)
    
    return familyc9_1, familyc9_2


In [77]:
familyc9_1, familyc9_2 = build_family_c9()
    
print("Number of graphs in familyc9_1:", len(familyc9_1))
print("Number of graphs in familyc9_2:", len(familyc9_2))
# Example: print one of them
for i in range(3):
    print(familyc9_1[i])
    print(familyc9_2[i])


Number of graphs in familyc9_1: 152
Number of graphs in familyc9_2: 152
Data(x=[7, 1], edge_index=[2, 24], y=[121], edge_labels=[304])
Data(x=[7, 1], edge_index=[2, 24], y=[121], edge_labels=[304])
Data(x=[9, 1], edge_index=[2, 32], y=[121], edge_labels=[304])
Data(x=[9, 1], edge_index=[2, 32], y=[121], edge_labels=[304])
Data(x=[11, 1], edge_index=[2, 40], y=[121], edge_labels=[304])
Data(x=[11, 1], edge_index=[2, 40], y=[121], edge_labels=[304])


In [78]:
def build_g1_edges(m: int):
    """
    Build the list of undirected edges for G1 (0-based).
    G1 has n = 2m nodes forming a cycle, plus an edge connecting (m-1) and (2m-1).
    """
    n = 2 * m
    edges = []
    
    for i0 in range(n):
        j0 = (i0 + 1) % n
        edges.append((i0, j0))
    
    edges.append((m - 1, 2*m - 1))
    undirected = []
    for (a, b) in edges:
        undirected.append((a, b))
        undirected.append((b, a))
    
    return undirected

def build_g2_edges(m: int):
    """
    Build the list of undirected edges for G2 (0-based).
    G2 has two cycles joined by a cut edge.
    """
    n = 2 * m
    edges = []
    
    for i0 in range(m):
        j0 = (i0 + 1) % m
        edges.append((i0, j0))
 
    for i0 in range(m):
        j0 = (i0 + 1) % m
        # shift by m
        edges.append((i0 + m, j0 + m))
    
    edges.append((m - 1, 2*m - 1))
    undirected = []
    for (a, b) in edges:
        undirected.append((a, b))
        undirected.append((b, a))
    
    return undirected

def make_edge_index_and_labels(edges, max_undirected=121):
    """
    Given a list of directed edges (a, b), buildedge_index, edge_labels.
    """

    edges = list(dict.fromkeys(edges))
    
    E = len(edges)
    # edge_index
    edge_index = torch.zeros((2, E), dtype=torch.long)
    for idx, (src, dst) in enumerate(edges):
        edge_index[0, idx] = src
        edge_index[1, idx] = dst
    edge_labels = torch.zeros(304, dtype=torch.long) # 304 for compatibility with c9
    
    return edge_index, edge_labels, edges

def build_family_c10():
    """
    Build two lists of Data objects: familyc10_1 (for G1) and familyc10_2 (for G2).
    m goes from 3 to 60 (inclusive).
    """
    familyc10_1 = []
    familyc10_2 = []
    
    max_num_nodes = 121   # =120 for m=60 but 121 for compatibility with c9 familites
    for m in range(3, 61):
        n = 2 * m
        
        # G1
        edges_g1 = build_g1_edges(m)
        edge_index_g1, edge_labels_g1, deduped_g1 = make_edge_index_and_labels(edges_g1)
        
        x_g1 = torch.tensor([[i] for i in range(n)], dtype=torch.long)
        # y all zeros for familyc10_1
        y_g1 = torch.zeros(max_num_nodes, dtype=torch.long)
        
        data_g1 = Data(
            x=x_g1,
            edge_index=edge_index_g1,
            y=y_g1,
            edge_labels=edge_labels_g1
        )
        familyc10_1.append(data_g1)
        
        # G2
        edges_g2 = build_g2_edges(m)
        edge_index_g2, edge_labels_g2, deduped_g2 = make_edge_index_and_labels(edges_g2)
        
        x_g2 = torch.tensor([[i] for i in range(n)], dtype=torch.long)
        
        # Set the (m-1) and (2m-1) entries in 0-based to 1
        y_g2 = torch.zeros(max_num_nodes, dtype=torch.long)
        y_g2[m-1] = 1
        y_g2[2*m - 1] = 1
        
        # Set the edge_labels
        edge_to_idx = {}
        for idx, (src, dst) in enumerate(deduped_g2):
            edge_to_idx[(src, dst)] = idx
        
        cut_edges = [
            (m - 1, 2*m - 1),
            (2*m - 1, m - 1),
        ]
        for (a, b) in cut_edges:
            if (a, b) in edge_to_idx:
                idx = edge_to_idx[(a, b)]
                if idx < 242:  # just a safety check
                    edge_labels_g2[idx] = 1
        
        data_g2 = Data(
            x=x_g2,
            edge_index=edge_index_g2,
            y=y_g2,
            edge_labels=edge_labels_g2
        )
        familyc10_2.append(data_g2)
    
    return familyc10_1, familyc10_2



In [79]:
familyc10_1, familyc10_2 = build_family_c10()
    
print("Number of graphs in familyc10_1:", len(familyc10_1))
print("Number of graphs in familyc10_2:", len(familyc10_2))
# Example: print one of them
for i in range(3):
    print(familyc10_1[i])
    print(familyc10_2[i])

Number of graphs in familyc10_1: 58
Number of graphs in familyc10_2: 58
Data(x=[6, 1], edge_index=[2, 14], y=[121], edge_labels=[304])
Data(x=[6, 1], edge_index=[2, 14], y=[121], edge_labels=[304])
Data(x=[8, 1], edge_index=[2, 18], y=[121], edge_labels=[304])
Data(x=[8, 1], edge_index=[2, 18], y=[121], edge_labels=[304])
Data(x=[10, 1], edge_index=[2, 22], y=[121], edge_labels=[304])
Data(x=[10, 1], edge_index=[2, 22], y=[121], edge_labels=[304])


In [80]:
max_edges = 0
for graph in familyc9_2:
    num_edges = graph.edge_index.shape[1]
    if num_edges > max_edges:
        max_edges = num_edges
print(max_edges)

304


In [81]:
biconn_dataset = rand_dataset + familyc9_1 + familyc9_2 + familyc10_1 + familyc10_2
print(len(biconn_dataset))


1008


In [82]:
# shuffle 
from random import shuffle
shuffle(biconn_dataset)

In [83]:
# save to file 
import pickle
# Save the list of torch data objects to a file
with open('biconn_dataset.pkl', 'wb') as f:
    pickle.dump(biconn_dataset, f)


In [6]:
with open('biconn_dataset.pkl', 'rb') as f:
    loaded_biconn_dataset = pickle.load(f)
print(loaded_biconn_dataset[0])

# reassign edge_labels to y to construct edges dataset
edges_dataset = []
for dat in loaded_biconn_dataset:
    new_dat = Data(
        x=dat.x,
        edge_index=dat.edge_index,
        y=dat.edge_labels
    )
    edges_dataset.append(new_dat)

print(edges_dataset[0])


Data(x=[28, 1], edge_index=[2, 84], y=[121], edge_labels=[304])
Data(x=[28, 1], edge_index=[2, 84], y=[304])


In [7]:
with open('biconn_edges_dataset.pkl', 'wb') as f:
    pickle.dump(edges_dataset, f)